In [1]:
import pandas as pd

# Specify the path to your Excel file
file_path = "Empty dwellings by canton.xls"

# Read the Excel file
# This reads all sheets into a dictionary with sheet names as keys and DataFrames as values
xlsx = pd.read_excel(file_path, sheet_name=None, skiprows=2)

# Check which sheets are present in the file
sheet_names = xlsx.keys()
print(f"Sheets found: {sheet_names}")




Sheets found: dict_keys(['2024 (basis BDS2023)', '2023 (basis BDS2022)', '2022 (basis BDS2021)', '2021 (basis BDS2020)', '2020 (basis BDS2019)', '2019 (basis BDS2018)', '2018 (basis BDS2017)', '2017 (basis BDS2016)', '2016 (basis BDS2015)', '2015 (basis BDS2014)', '2014 (basis BDS2013)', '2013 (basis BDS2012)', '2012 (basis BDS2011)', '2011 (new basis BDS2010)', '2010 (new basis BDS2009)', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999'])


In [9]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import re
import logging

# Configure logging to display information and error messages
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def find_header_row(df, keywords=["Total dwelling stock", "Total Dwellings"]):
    """
    Finds the row index where the header starts based on a list of keywords.

    Parameters:
    - df (pd.DataFrame): The raw DataFrame read without headers.
    - keywords (list): A list of keywords to identify the header row.

    Returns:
    - int or None: The index of the header row, or None if not found.
    """
    for i in range(len(df)):
        row = df.iloc[i].astype(str).str.lower()
        for keyword in keywords:
            if row.str.contains(keyword.lower()).any():
                return i
    return None  # Return None if no header row is found

def find_header_row_enhanced(df, sheet_name, manual_header_sheets, default_keywords=["Total dwelling stock", "Total Dwellings"]):
    """
    Finds the header row index and number of header rows, using manual specifications if available.

    Parameters:
    - df (pd.DataFrame): The raw DataFrame read without headers.
    - sheet_name (str): The name of the sheet being processed.
    - manual_header_sheets (dict): Dictionary mapping sheet names to (header_row, num_header_rows).
    - default_keywords (list): Keywords to identify the header row.

    Returns:
    - tuple: (header_row, num_header_rows) or (None, None) if not found.
    """
    if sheet_name in manual_header_sheets:
        header_info = manual_header_sheets[sheet_name]
        logging.info(f"Using manual header row {header_info[0]} with {header_info[1]} header row(s) for sheet '{sheet_name}'.")
        return header_info
    else:
        header_row = find_header_row(df, keywords=default_keywords)
        if header_row is not None:
            logging.info(f"Automatically detected header row at index {header_row} for sheet '{sheet_name}'.")
            return (header_row, 4)  # Default to 4 header rows for automated sheets
        else:
            logging.warning(f"No header row detected for sheet '{sheet_name}'.")
            return (None, None)

def combine_header_rows(df, header_start, num_header_rows=4, join_separator=' '):
    """
    Combines multiple header rows into a single header by concatenating the text.

    Parameters:
    - df (pd.DataFrame): The raw DataFrame read without headers.
    - header_start (int): Starting row index for headers (inclusive).
    - num_header_rows (int): Number of header rows to combine.
    - join_separator (str): Separator to use when joining header strings.

    Returns:
    - pd.DataFrame: DataFrame with combined headers and header rows removed.
    - list: List of combined header strings.
    """
    # Ensure we don't exceed the DataFrame's length
    header_end = header_start + num_header_rows
    header_end = min(header_end, len(df))

    # Extract header rows
    header_data = df.iloc[header_start:header_end].fillna('')  # Replace NaN with empty strings

    # Combine header rows for each column by concatenating non-empty strings
    combined_headers = header_data.apply(
        lambda x: join_separator.join([str(item).strip() for item in x if str(item).strip() != '']),
        axis=0
    )

    # Assign combined headers to the DataFrame
    df.columns = combined_headers

    # Remove the header rows from the DataFrame
    df_cleaned = df.iloc[header_end:].reset_index(drop=True)

    return df_cleaned, combined_headers.tolist()

def clean_dataframe_enhanced(df, combined_headers, sheet_name, manual_header_sheets, expected_columns=None):
    """
    Cleans the DataFrame by removing empty columns, handling NaN values, and ensuring expected columns.

    Parameters:
    - df (pd.DataFrame): The DataFrame to clean.
    - combined_headers (list): List of combined header strings.
    - sheet_name (str): Name of the current sheet.
    - manual_header_sheets (dict): Dictionary mapping sheet names to (header_row, num_header_rows).
    - expected_columns (list): List of expected column names to retain.

    Returns:
    - pd.DataFrame: Cleaned DataFrame.
    """
    # Remove columns where the combined header is empty
    non_empty_columns = [col for col in df.columns if col.strip() != '']
    df_cleaned = df[non_empty_columns].copy()

    # Reset index after removing columns
    df_cleaned.reset_index(drop=True, inplace=True)

    # Handle NaN values in numeric columns
    numeric_cols = df_cleaned.select_dtypes(include=['number']).columns
    df_cleaned[numeric_cols] = df_cleaned[numeric_cols].fillna(0)

    # Handle categorical columns (e.g., Region)
    # Identify the first non-numeric column as 'Region' or similar
    non_numeric_cols = df_cleaned.select_dtypes(exclude=['number']).columns.tolist()
    if non_numeric_cols:
        primary_cat_col = non_numeric_cols[0]
        df_cleaned[primary_cat_col] = df_cleaned[primary_cat_col].fillna('Unknown')
        # Optionally, rename the column to 'Region' if it contains 'region' or 'total'
        if 'region' in primary_cat_col.lower() or 'total' in primary_cat_col.lower():
            df_cleaned.rename(columns={primary_cat_col: 'Region'}, inplace=True)
    else:
        logging.warning(f"No categorical columns found in sheet '{sheet_name}'. Consider verifying the header rows.")

    # For manual sheets, assign column names manually
    if sheet_name in manual_header_sheets:
        # Define the expected column names based on your Excel sheet structure
        manual_columns = [
            'Region',
            'Total dwelling stock',
            'Empty dwellings with 1 room',
            'Empty dwellings with 2 rooms',
            'Empty dwellings with 3 rooms',
            'Empty dwellings with 4 rooms',
            'Empty dwellings with 5 rooms',
            'Empty dwellings with 6+ rooms',
            'Total Empty Dwellings',
            'Of which',
            'Proportion of empty dwelling 2)'
        ]
        
        # Check if the number of columns matches
        if len(manual_columns) == len(df_cleaned.columns):
            df_cleaned.columns = manual_columns
        else:
            logging.warning(f"Manual column assignment mismatch for sheet '{sheet_name}'. Expected {len(manual_columns)} columns, found {len(df_cleaned.columns)}.")
    else:
        # Optionally, retain only expected columns for automated sheets
        if expected_columns:
            existing_expected_cols = [col for col in expected_columns if col in df_cleaned.columns]
            df_cleaned = df_cleaned[existing_expected_cols]

    return df_cleaned

def add_total_row(df, first_column='Region'):
    """
    Adds a 'Total' row to the DataFrame by summing numeric columns.

    Parameters:
    - df (pd.DataFrame): The DataFrame to process.
    - first_column (str): The name of the first descriptive column.

    Returns:
    - pd.DataFrame: The DataFrame with the 'Total' row appended.
    """
    try:
        # Identify numeric columns
        numeric_cols = df.select_dtypes(include=['number']).columns.tolist()

        # Calculate sums
        sum_values = df[numeric_cols].sum()

        # Create a dictionary for the Total row
        total_dict = {col: sum_values[col] for col in numeric_cols}
        total_dict[first_column] = 'Total'

        # Create a DataFrame from the total_dict
        total_df = pd.DataFrame([total_dict])

        # Concatenate the Total row
        df_with_total = pd.concat([df, total_df], ignore_index=True)

        return df_with_total

    except Exception as e:
        logging.error(f"An error occurred while adding Total row: {e}")
        return df

def process_excel_sheets_enhanced_v2(file_path, sheet_names, manual_header_sheets, keywords=["Total dwelling stock", "Total Dwellings", "total dwelling stock"], num_header_rows_default=4, join_separator=' '):
    """
    Processes multiple sheets in an Excel file, cleaning and organizing the data with enhanced header detection and cleaning.

    Parameters:
    - file_path (str): Path to the Excel file.
    - sheet_names (list): List of sheet names to process.
    - manual_header_sheets (dict): Dictionary mapping sheet names to (header_row, num_header_rows).
    - keywords (list): List of keywords to identify the header row.
    - num_header_rows_default (int): Default number of header rows for automated sheets.
    - join_separator (str): Separator to use when joining header strings.

    Returns:
    - dict: Dictionary of cleaned DataFrames keyed by sheet name.
    """
    cleaned_data = {}

    for sheet in sheet_names:
        logging.info(f"\nProcessing Sheet: {sheet}")

        # Read the sheet without headers
        try:
            df_raw = pd.read_excel(file_path, sheet_name=sheet, header=None, engine='xlrd')
            logging.info(f"Sheet '{sheet}' read successfully. Shape: {df_raw.shape}")
        except Exception as e:
            logging.error(f"Failed to read sheet '{sheet}': {e}")
            continue

        # Detect the header row and number of header rows
        header_info = find_header_row_enhanced(df_raw, sheet, manual_header_sheets, default_keywords=keywords)
        header_row, num_header_rows = header_info

        if header_row is None:
            logging.warning(f"No header row found in sheet '{sheet}'. Skipping this sheet.")
            continue

        logging.info(f"Detected header row at index: {header_row}")

        # Combine header rows
        df_cleaned, combined_headers = combine_header_rows(df_raw, header_start=header_row, num_header_rows=num_header_rows, join_separator=join_separator)
        logging.info("Combined Headers:")
        for idx, header in enumerate(combined_headers):
            logging.info(f"Column {idx}: '{header}'")

        # Define expected columns based on combined_headers
        expected_columns = [header for header in combined_headers if 'municipality' in header.lower() or 'total' in header.lower() or 'rooms' in header.lower() or 'proportion' in header.lower()]
        if not expected_columns and sheet not in manual_header_sheets:
            # Fallback: retain all non-empty columns
            expected_columns = [header for header in combined_headers if header.strip() != '']

        # Clean the DataFrame with enhanced cleaning
        df_cleaned = clean_dataframe_enhanced(df_cleaned, combined_headers, sheet, manual_header_sheets, expected_columns=expected_columns)
        logging.info("Cleaned DataFrame Preview:")
        logging.info(df_cleaned.head())

        # Add 'Year' column based on sheet name
        year_match = re.search(r'\d{4}', sheet)
        if year_match:
            year = year_match.group()
        else:
            year = 'Unknown'

        df_cleaned['Year'] = year
        logging.info(f"Assigned Year: {year}")

        # Reorder columns to have 'Year' first
        cols = df_cleaned.columns.tolist()
        cols = ['Year'] + [col for col in cols if col != 'Year']
        df_cleaned = df_cleaned[cols]

        # Add 'Total' row
        if sheet in manual_header_sheets:
            total_label_column = 'Region'
        else:
            total_label_column = expected_columns[0] if expected_columns else 'Unknown'
        df_cleaned = add_total_row(df_cleaned, first_column=total_label_column)
        logging.info("DataFrame with Total row:")
        logging.info(df_cleaned.tail())

        # Store the cleaned DataFrame
        cleaned_data[sheet] = df_cleaned

    return cleaned_data

# Define the path to your Excel file
file_path = 'Empty dwellings by canton.xls'

# List of sheet names to process
sheet_names = [
    '2024 (basis BDS2023)', '2023 (basis BDS2022)', '2022 (basis BDS2011)',
    '2021 (basis BDS2020)', '2020 (basis BDS2019)', '2019 (basis BDS2018)',
    '2018 (basis BDS2017)', '2017 (basis BDS2016)', '2016 (basis BDS2015)',
    '2015 (basis BDS2014)', '2014 (basis BDS2013)', '2013 (basis BDS2012)',
    '2012 (basis BDS2011)', '2011 (new basis BDS2010)', '2010 (new basis BDS2009)',
    '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002',
    '2001', '2000', '1999'
]

# Define sheets that require manual header row specification with (header_row, num_header_rows)
manual_header_sheets = {
    '2007': (52, 1),
    '2006': (52, 1),
    '2005': (52, 1),
    '2004': (52, 1),
    '2003': (52, 1),
    '2002': (3, 3),   # Updated for '2002' sheet
    '2001': (52, 1),
    '2000': (52, 1),
    '1999': (52, 1)
    # Add any other sheets requiring manual header rows here
}

# Process all sheets with enhanced processing
cleaned_data = process_excel_sheets_enhanced_v2(
    file_path=file_path,
    sheet_names=sheet_names,
    manual_header_sheets=manual_header_sheets,
    keywords=["Total dwelling stock", "Total Dwellings", "total dwelling stock"],
    num_header_rows_default=4,      # Default number of header rows for automated sheets
    join_separator=' '               # Separator used when combining header rows
)

# Example Access to a Cleaned DataFrame
# Replace '2002' with the actual sheet name you want to inspect
sheet_to_inspect = '2002'  # Ensure this matches exactly with the sheet name

if sheet_to_inspect in cleaned_data:
    df_2002 = cleaned_data[sheet_to_inspect]
    logging.info(f"\nDataFrame for '{sheet_to_inspect}':")
    print(df_2002)
else:
    logging.warning(f"Sheet '{sheet_to_inspect}' not found in cleaned_data.")


INFO: 
Processing Sheet: 2024 (basis BDS2023)
INFO: Sheet '2024 (basis BDS2023)' read successfully. Shape: (45, 14)
INFO: 
Processing Sheet: 2023 (basis BDS2022)
INFO: Sheet '2023 (basis BDS2022)' read successfully. Shape: (45, 14)
INFO: 
Processing Sheet: 2022 (basis BDS2011)
ERROR: Failed to read sheet '2022 (basis BDS2011)': Worksheet named '2022 (basis BDS2011)' not found
INFO: 
Processing Sheet: 2021 (basis BDS2020)
INFO: Sheet '2021 (basis BDS2020)' read successfully. Shape: (60, 30)
INFO: Automatically detected header row at index 3 for sheet '2021 (basis BDS2020)'.
INFO: Detected header row at index: 3
INFO: Combined Headers:
INFO: Column 0: ''
INFO: Column 1: ''
INFO: Column 2: 'Total dwelling stock on the 31.12.2020  1)'
INFO: Column 3: ''
INFO: Column 4: 'Empty dwellings with … rooms 1'
INFO: Column 5: ''
INFO: Column 6: '2.0'
INFO: Column 7: ''
INFO: Column 8: '3.0'
INFO: Column 9: ''
INFO: Column 10: '4.0'
INFO: Column 11: ''
INFO: Column 12: '5.0'
INFO: Column 13: ''
INFO

    Year Empty dwellings with … rooms Proportion of empty dwellings 2) Region
0   2002                            1                              NaN    NaN
1   2002                      Unknown                              NaN    NaN
2   2002                      Unknown                              NaN    NaN
3   2002                      Unknown                              NaN    NaN
4   2002                      Unknown                              NaN    NaN
5   2002                         5062                             1.04    NaN
6   2002                      Unknown                              NaN    NaN
7   2002                         1143                         0.768247    NaN
8   2002                          489                         0.746401    NaN
9   2002                          533                         1.382541    NaN
10  2002                          121                          0.24993    NaN
11  2002                      Unknown                           

In [10]:
df_2002

,Year,Empty dwellings with … rooms,Proportion of empty dwellings 2),Region
0,2002,1,NaN,NaN
1,2002,Unknown,NaN,NaN
2,2002,Unknown,NaN,NaN
3,2002,Unknown,NaN,NaN
4,2002,Unknown,NaN,NaN
5,2002,5062,1.04,NaN
6,2002,Unknown,NaN,NaN
7,2002,1143,0.768247,NaN
8,2002,489,0.746401,NaN
9,2002,533,1.382541,NaN
